In [1]:
import wandb
import pandas as pd
api = wandb.Api()

# run = api.run("<entity>/<project>/<run_id>")
# run.config["key"] = updated_value
# run.update()
# api.runs("project_path_here", filters={"createdAt": {"$in": ["date"]}})
runs = api.runs("q191106702/CVR prediction test", 
                filters={"$or":[
                            {"tags": {"$in": ["src.models.descm.DESCM_Embedding_DA_1_1_Base"]}},
                            {"tags": {"$in": ["src.models.descm.DESCM_Embedding_DA_1_1"]}},
                            {"tags": {"$in": ["src.models.descm.DESCM_Embedding_DA_2_1"]}},      
                            ]
                         ,
                        #  "$and":[
                        #     {"tags": {"$in": ["fr"]}}     
                        #     ]
                         }
                )

# Assume runs is your list of runs
data_list = [
    {'test/cvr_auc': run.summary._json_dict.get('test/cvr_auc', None),
    'test/ctcvr_auc': run.summary._json_dict.get('test/ctcvr_auc', None),
    'test/ctr_auc': run.summary._json_dict.get('test/ctr_auc', None),
    'tags': run.tags} for run in runs]

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(data_list)
df = df.round(3)

# Function to extract tag from list
def extract_tag(row):
    for tag in tags_to_extract:
        if tag in row['tags']:
            return tag
    return None

def extract_tag_as_new_column(df, tag_to_extract, tag_name='data'):
    df[tag_name] = df.apply(extract_tag, axis=1)
    df['tags'] = [[tag for tag in tags if tag not in tags_to_extract] for tags in df['tags']]
    return df
    
    

# set data column
tags_to_extract = ['fr', 'es', 'nl', 'us', 'ccp']
df = extract_tag_as_new_column(df, tags_to_extract, 'data')
#set the method column
tags_to_extract = ['src.models.descm.DESCM_Embedding_DA_1_1_Base', 'src.models.descm.DESCM_Embedding_DA_1_1','src.models.descm.DESCM_Embedding_DA_2_1']
df = extract_tag_as_new_column(df, tags_to_extract, 'methods')
df['methods'] = df['methods'].str.split('.').str[-1]


# Convert 'tags' column to string
df['tags'] = df['tags'].apply(lambda x: ','.join(x))
# Split df into df_cvr, df_ctcvr, df_ctr_auc
df_cvr = df[['test/cvr_auc', 'data', 'methods']]
df_ctcvr = df[['test/ctcvr_auc', 'data','methods']]
df_ctr = df[['test/ctr_auc', 'data', 'methods']]

# Pivot each DataFrame
df_cvr = df_cvr.groupby(['methods', 'data']).first().reset_index()
df_cvr = df_cvr.groupby(['methods', 'data']).first().reset_index()

df_cvr = df_cvr.pivot(index='methods', columns='data', values='test/cvr_auc').reset_index()
df_ctcvr = df_ctcvr.groupby(['methods', 'data']).first().reset_index()
df_ctcvr = df_ctcvr.pivot(index='methods', columns='data', values='test/ctcvr_auc').reset_index()
df_ctr = df_ctr.groupby(['methods', 'data']).first().reset_index()
df_ctr = df_ctr.pivot(index='methods', columns='data', values='test/ctr_auc').reset_index()

# save *_cvr as csv
df_cvr.to_csv('cvr.csv', index=False)
df_ctcvr.to_csv('ctcvr.csv', index=False)
df_ctr.to_csv('ctr_auc.csv', index=False)
# Merge the DataFrames on 'method'
df_combined = df_cvr.merge(df_ctcvr, on='methods', suffixes=('_cvr', '_ctcvr'))
df_combined = df_combined.merge(df_ctr, on='methods')

# df_combined.rename(columns={'ccp': 'ccp_ctr', 'fr':'fr_ctr','nl':'nl_ctr','us':'us_ctr','es':'es_ctr'}, inplace=True)
df_combined.to_csv('combined.csv', index=False)
df_combined

data,methods,ccp_cvr,es_cvr,fr_cvr,nl_cvr,us_cvr,ccp_ctcvr,es_ctcvr,fr_ctcvr,nl_ctcvr,us_ctcvr,ccp_ctr,es_ctr,fr_ctr,nl_ctr,us_ctr
0,DESCM_Embedding_DA_1_1,0.553,0.650,0.617,0.590,0.647,0.564,0.799,0.771,0.764,0.765,0.621,0.719,0.715,0.715,0.702
1,DESCM_Embedding_DA_1_1_Base,0.673,0.805,0.802,0.768,0.798,0.640,0.867,0.858,0.841,0.854,0.620,0.718,0.712,0.717,0.700
2,DESCM_Embedding_DA_2_1,0.652,0.746,0.765,0.718,0.732,0.613,0.838,0.837,0.807,0.811,0.621,0.721,0.716,0.719,0.698
